### Libraries

In [1]:
# Loading data
import pickle

# Graph
import networkx as nx

# Plots
import dash
from dash import Dash, dcc, html, Input, Output

# Invocation functions
from functions_plot import *

## Load data

In [2]:
# Element profiles
with open('3. element.pkl', 'rb') as f: 
    element_profiles = pickle.load(f) 

In [3]:
# Element dependencies
with open('4. dependencies.pkl', 'rb') as f: 
    element_dependencies = pickle.load(f) 

### Create Graph structure

In [4]:
#-----------------------------
# Create empty graph
G = nx.DiGraph()

#-----------------------------
# node_list = []

# Add nodes
for node in element_profiles:
    
    # Fetch node properties:
    

    file_name  = node['file_name']
    file_location = node['file_location']
    file_lines = node['file_line_number']
    node_name = node['element_name'].lower()
    node_type = node['element_type']
    node_code = node['element_code']
    node_attributes = node['Values'] 
    node_id = file_name + '|' + node_name
        

    
    # check if node already exists:
    if G.has_node(node_id):
        # fetch code element
        node_code_prev = G.nodes[node_id]['node_code']
        # Extend code element
        G.nodes[node_id]['node_code'] = node_code_prev + '\n' + node_code
        
    # otherwise create node:
    else:
    
        # Get optional attributes:
        if node_type == 'Code':
            element_input = ''
            element_comments = ''
            element_output = ''
        elif node_type == 'Function' or node_type == 'Class':    
            element_input = node_attributes['Input_var']
            element_comments = node_attributes['Code_comments']
            element_output = node_attributes['Output_var']

        # output results
        G.add_node(node_id,
                   # Attributes
                   file_name = file_name,
                   file_location = file_location,
                   file_lines = file_lines,
                   node_name = node_name,
                   node_type = node_type,
                   node_code = node_code,
                   attributes_input = element_input,
                   attributes_comments = element_comments,
                   attributes_output = element_output
                  )      
    
#     # Store node list
#     if node_id not in node_list:
#         node_list.append(node_id)
        
#-----------------------------
# Add edges to graph
for edges in element_dependencies:
    
    # Fetch
    edge_from = edges['function_from'].lower()
    edge_to = edges['function_to'].lower()
    file_from = edges['file_from']
    file_to = edges['file_to']
    edge_from_id = file_from + '|' + edge_from
    edge_to_id = file_to + '|' + edge_to   
    
        
    # check if edge already exists
    if G.has_edge(edge_from_id, edge_to_id):
        # Fetch attributes
        attribute = G[edge_from_id][edge_to_id]["invocation"]
        # Extend values
        attribute.append(edges['values']) 
        # Modify edge
        G[edge_from_id][edge_to_id]["invocation"] = attribute
        
    else:
        # Create edge
        G.add_edge(edge_from_id, edge_to_id, 
                   # Attributes
                   invocation = [edges['values']],
                   edge_from = edge_from,
                   edge_to = edge_to,
                   file_from = file_from,
                   file_to = file_to
                  )
        

# Add positions to the graph
pos = get_coordinates(G)
nx.set_node_attributes(G, pos, 'pos') 

### Dash plot

In [5]:
# network trace
edge_trace = get_edge_trace(G)
node_trace = get_node_trace(G)

fig_net = plot_graph(edge_trace, node_trace)

In [6]:
# Get Table
df = get_code_table(G)

# Table trace
fig_table  = draw_table(df)

In [8]:
# Server start
app = Dash()

#======================================================================
# Dash plot
app.layout = html.Div(children = 
                      [# Reset button
                          html.Button('Reset Graphs', 
                                      id='reset_button', 
                                      n_clicks=0,
                                      style={'height': '3vh'}), 
                       # Plot
                          dcc.Graph(id='network', 
                                 figure=fig_net,
                                 style={'width': '100vw',
                                       'height': '43vh'}),
                       
                       # Table
                          fig_table                     
                      ])                   
    
#======================================================================
# Callbacks
@app.callback([Output('reset_button','n_clicks'), Output('table', 'filter_query'), Output('network', 'selectedData'), Output('network', 'figure')],
              [Input('reset_button','n_clicks'), Input('table', 'filter_query'), Input('network', 'selectedData')])
def update_figures(n_clicks, filter_query, selectedData):
    
    # 1. If reset is clicked - restart table and scatter
    if n_clicks is not None and n_clicks > 0:  
        return 0, '', None, fig_net  
    
    # 2. If filter is applied to Network -> update Table
    elif selectedData is not None:
        selected_query = update_query(selectedData)
        if filter_query != '':            
            full_query = '(' + filter_query + ') and (' + selected_query + ')' # Data selected based on query and scatter
            return dash.no_update, full_query, dash.no_update, dash.no_update
        else:
            return dash.no_update, selected_query, None, dash.no_update
        
    # 3. If filter is applied to Table - update Network
    elif filter_query is not None and filter_query != '': 
        fig_updated = update_scatter(filter_query, fig_net, df)
        return dash.no_update, dash.no_update,  None, fig_updated
    
    # 4. If filter query is reset - update Network
    elif filter_query == '' and selectedData is None:
        return dash.no_update, dash.no_update,  None, fig_net        
    
    # 5. No change
    else:
        raise dash.exceptions.PreventUpdate

#======================================================================
app.run_server(debug=False)    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:34] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:05:35] "POST /_dash-update-component HTTP/1.1" 200 -


Table -> Scatter
{Name} scontains main


127.0.0.1 - - [13/Apr/2023 15:05:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 15:06:19] "POST /_dash-update-component HTTP/1.1" 200 -


Table -> Scatter
{Name} scontains "Name: main.py"
